In [1]:
import pandas as pd
import numpy as np
import statistics
import datetime
import matplotlib.pyplot as plt
import re

In [2]:
raw = pd.read_csv("data.csv")
# set(raw.columns)

/Users/eunicheng/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (14,20,28,31,32,56,68,69,72,73,76,77,78,80,81,82,83,84,86,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#刪掉無資訊、明顯無相關的欄位
saved_columns = [
 'KeyNo',
 'TypeIdName', #('其他', '犬', '貓')
 'AcceptDate', #被領養日期？
 'DistrictTeamName',
 'SexName',#{'none', '公', '未知', '母'}
 'ReasonName',#{nan, '不擬續養', '依法沒入', '其他', '動物救援', '拾獲送交', '收容所轉入', '政府捕捉'}
 'VarietyName',#種類，阿富汗、馬爾濟斯...71種，其中兩個是數字268,269
 'VarietyRemark',#更細的種類，沒有救NaN
 'BuildName',#體型名稱{'1', '中', '大', '小'}
 'Bodyweight',#超亂，setlen=12135, nan, NaN, 13~15, 2公斤, 4....
 'Age2Name',#{'1至3月齡', '1至3歲', '3至6月齡', '3至7歲', '6月齡至1歲', '7歲以上', nan, 'none', '未離乳'}
 'CoatName',#毛色，28種，包括'none'
#  'MouthCoat',#亂，nan,'string', '鼻上方有一咖啡色胎記', '黑嘴','黑嘴    '
#  'HeadCoat',# 亂，set多個nan???
#  'ChestCoat',#亂，nan, 'string'...
#  'FootCoat',#亂，nan, 'string'...
#  'EarCoat',# 亂，set多個nan???
#  'BackCoat',# 亂，set多個nan???
#  'TailCoat',#亂，nan,'string',
 'CollarName',#項圈顏色{'none', '其他', '棕色', '無', '白色', '粉紅色', '紅色', '花色', '藍色', '銀色', '黃色', '黑色'}
#  'CollarRemark',#項圈特徵
 'ThoracodorsalName',#胸甲(衣服)顏色{'none', '其他', '棕色', '無', '白色', '紅色', '花色', '藍色', '黃色', '黑色'}
#  'ThoracodorsalRemark',#多，胸甲(衣服)特徵
 'AdoptionName',#{'待認領', '暫時不適合', '適合認養'}
 'NotAdoptionName',#{'none', '健康不適', '動保案件', '年紀不適', '行為不適'}
 '_Situation2Name',#{'none','人道處理其他法令授權','人道處理第12條第1項第3款','人道處理第12條第1項第5款', '人道處理第12條第1項第7款', '其他','原地飭回','收容所轉出','生理耗弱死亡','疾病死亡','認領','認養','逃脫'}
#  'SituationRemark',#認養狀態備註
 'StatusName',#{'sNull', '其他', '原地飭回', '安樂死', '生理耗弱死亡', '疾病死亡', '認領養', '逃跑'}
#  'Address',
 'IsSterilizationName',#{'是', '未確定'}
#  'SterilizationDate',
 'EarNoteName',#{'none', '右剪耳', '左剪耳'}
#  'Note',#超雜
 'LengthCoatName',#{'none', '中長毛', '未修剪過', '短毛', '長毛'}
 'HealthStatusName',#{'none', '其它', '寄生蟲', '正常', '異常', '疫苗'}
#  'HealthStatusRemark',#超多受傷原因
 'Behavior',#幾種數字，但還不知道數字代表的意思
#  'BehaviorRemark',#超多行為描述
#  'InImage',
#  'ImageNameStr',
#  'FrontImageName',
#  'VaccineDate', #沒填示沒打過疫苗？
 'ShelterName',
#  'VillageID',
#  'VillageName',
 'FinalDate'#領養日期
]
raw = raw[saved_columns]

In [4]:
#dropna columns
dropna_col = ['FinalDate','ReasonName']
# replacenan_col = ['DistrictTeamName', 'VarietyRemark','Bodyweight',
#                   'MouthCoat','HeadCoat','ChestCoat','FootCoat','EarCoat',
#                   'BackCoat','TailCoat','HealthStatusRemark','BehaviorRemark','CollarRemark',
#                   'ThoracodorsalRemark','SituationRemark','Behavior','VillageName']
print("Before rm na:", raw.shape)

for d_col in dropna_col:
    raw=raw[raw[d_col].notna()]
    print(d_col, raw.shape)
raw.fillna('無', inplace=True)
raw.replace('none', '無', inplace=True)
raw['StatusName'].replace('sNull', '無', inplace=True)
print("Aefore rm na:", raw.shape)

Before rm na: (115267, 25)
FinalDate (115267, 25)
ReasonName (115261, 25)
Aefore rm na: (115261, 25)


處理 '_Situation2Name'  

1：'認養'  

0:'無'  

刪掉：'人道處理其他法令授權','人道處理第12條第1項第3款','人道處理第12條第1項第5款', '人道處理第12條第1項第7款','其他','原地飭回','收容所轉出','認領','逃脫','生理耗弱死亡','疾病死亡'

#{'none','人道處理其他法令授權','人道處理第12條第1項第3款','人道處理第12條第1項第5款', '人道處理第12條第1項第7款', '其他','原地飭回','收容所轉出','生理耗弱死亡','疾病死亡','認領','認養','逃脫'}

In [5]:
tmp = [True if x in ['無', '認養'] else False for x in raw['_Situation2Name']]
raw = raw[tmp]
tmp = [False if x == "測試收容所" else True for x in raw['ShelterName']]
raw = raw[tmp]
raw['Adopted'] = [1 if x == "認養" else 0 for x in raw['_Situation2Name']]
print('# of adopted: {} \n# of not adopted:{}'.format(sum(raw['Adopted']), len(raw['Adopted'])-sum(raw['Adopted'])))


# of adopted: 62837 
# of not adopted:8020


In [6]:
#[Staydays]
raw['AcceptDate'] = [datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S") for x in raw['AcceptDate']] 
raw['FinalDate'] = [datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S") for x in raw['FinalDate']] 
print(len(raw))
tmp = [False if x == datetime.datetime(1, 1, 1, 0, 0) else True for x in raw['AcceptDate']]
raw = raw[tmp]
print(len(raw))

raw["StayDays"] = raw["FinalDate"]-raw['AcceptDate']
raw['StayDays'] = [x.days for x in raw['StayDays']] 
#[Staydays] 清掉未被領養的資料
# raw = raw[raw['StayDays']>=0]
raw['StayDays'] = ["未被領養" if x < 0 else x for x in raw['StayDays']]
raw.drop(columns=['AcceptDate', 'FinalDate'], inplace=True)

#[SexName]clean
raw["SexName"].replace({"none": "未知"}, inplace=True)
#VarietyName=>268,269清掉？

#[BuildName]clean
raw = raw[raw["BuildName"] != '1']
#[VarietyRemark]clean
tmp = [False if x in ['0','MIX','mix','string'] else True for x in raw['VarietyRemark']]
raw = raw[tmp]

print("After cleaning dirty data:", raw.shape)


70857
70837
After cleaning dirty data: (70834, 25)


In [7]:
# data = [x for x in raw['StayDays'] if x  != "未被領養"]
# plt.hist(data)
# sorted(raw['AcceptDate'])
# sum([True if x == datetime.datetime(1, 1, 1, 0, 0) else False for x in raw['AcceptDate']])

In [8]:
#[Bodyweight]clean
tmp = [False if x in ['strin','東22','１',''] else True for x in raw['Bodyweight']]
raw = raw[tmp]
raw['Bodyweight'] = [str(x).replace('K', '').replace('G', '').replace('k', '').replace('g', '').replace('<', '').replace('-', '~').replace('ｋｇ', '').replace('公斤', '').replace('以上', '').replace('以下', '').replace('體重', '').replace('胖', '20').strip(". ") for x in raw['Bodyweight']]
tmp = []
for x in raw['Bodyweight']:
        if x == "無" or x == '':
            tmp.append("無")
            continue
        if x.find("~") != -1:
            tmp.append(statistics.mean([float(y) for y in x.split("~")]))
        else:
            tmp.append(float(x))
raw['Bodyweight'] = tmp
#[Bodyweight]remove outlier
tmp = np.array([x for x in tmp if x != "無"])
Q1= np.mean(tmp)-2*np.std(tmp)
Q3= np.mean(tmp)+2*np.std(tmp)
raw = raw[[True if x == "無" or (x >= Q1 and x <= Q3) else False for x in raw['Bodyweight']]]
#[Bodyweight]convert to category(0~5, 5~10, 10~15...20以上)
tmp = []
for x in raw['Bodyweight']:
    if x == "無":
        tmp.append("無")
    elif x > 0 and x <= 5:
        tmp.append("0~5")
    elif x > 5 and x <= 10:
        tmp.append("5~10")
    elif x > 10 and x <= 15:
        tmp.append("10~15")
    elif x > 15 and x <= 20:
        tmp.append("15~20")
    else:
        tmp.append("20以上")
raw['Bodyweight'] = tmp

print("After cleaning dirty data:", raw.shape)

After cleaning dirty data: (70816, 25)


In [9]:
tmp = [True if x not in ["年老", "廠商測試","品種說明","銀","白底，頭尾有花色"] else False for x in raw['VarietyRemark']]
raw = raw[tmp]
raw["VarietyRemark"] = [s.strip().replace("混", "").replace("似", "").replace(" ", "").replace('可能到','').replace("有到","").replace("很像","").replace("微","").replace("疑到","").replace("類","") for s in raw['VarietyRemark']]
raw['VarietyRemark'] = ["種犬" if (x == "種品種犬" or x == "種狗") else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["種貓" if (x == "種貓(美短+豹貓)") else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = [x if (x == "種犬" or x == "種貓") else x.replace("種","") for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["暹羅貓" if x == "暹羅" else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["柴犬" if x == "柴" else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["拉不拉多" if x == "拉不拉" else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["梗犬" if (x == "梗" or x == "梗血統") else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["鵝" if x == "鵝4隻" else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["和尚鸚鵡" if x == "和尚" else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["查理士" if x == "查理氏" else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["喜馬拉雅貓" if x == "喜馬拉雅" else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["比特犬" if x == "比特" else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["鬥牛犬" if x == "鬥牛" else x for x in raw['VarietyRemark']]
raw['VarietyRemark'] = ["高加索犬" if x == "高加索" else x for x in raw['VarietyRemark']]

tmp = [True if x not in ["","黑色","美短","小型"] else False for x in raw['VarietyRemark']]
tmp = [True if x!="" else False for x in raw['VarietyRemark']]
raw = raw[tmp]
print(set(raw['VarietyRemark']))



{'柯基犬', '大麥町', '高山犬', '長毛臘腸犬', '查理士', '美國惡霸', '靈緹', '異國短毛貓', '狼犬哈士奇', '孟加拉豹貓', '種貓', '黑色貴賓', '鴿子', '巴及度', '迷你杜檳', '蛇', '德國狼犬', '無', '梗犬', '挪威森林貓', '黑色', '喜樂蒂', '吉娃娃約克夏', '兔子', '鴨', '哈士奇', '白鼻心', '黑貴賓', '杜賓', '鬥牛犬', '米格魯', '高山比特', '種犬', '高加索犬', '比特犬', '金吉拉', '奶油貴賓', '蜜袋鼯', '秋田', '刺蝟', '小型', '小杜賓', '鵝', '羊', '鸚鵡', '可麗牧羊犬', '貴賓', '松鼠', '吸蜜鸚鵡', '蝴蝶犬+吉娃娃', '波斯', '水牛', '小太陽', '瑪加吉娃娃', '獒犬', '日本狆', '高砂犬', '比特波士頓犬', '西施', '波音達', '牧羊犬', '和尚鸚鵡', '藍貓', '加菲貓', '土狼犬', '蘇格蘭梗', '短毛臘腸', '長毛', '黑梗犬', '牛頭梗', '臘腸', '紅貴賓', '美短', '米克斯', '俄國藍貓', '尋血犬', '潘布魯克', '比利時狼犬', '吉娃娃', '喜馬拉雅貓', '柴犬', '麝香豬', '狐狸犬', '長毛吉娃娃', '赫尼亞', '孔雀', '拉不拉多', '迷你杜賓', '雪納瑞', '貂', '馬爾濟斯', '三花', '法鬥貴賓', '迷你品', '狼犬', '凱門鱷', '長毛臘腸', '暹羅貓', '累杜賓犬', '黑豬'}


# Coat 如何切分特徵，同時有多個特徵？

### 橘貓很受歡迎？？？

In [10]:
# #ChestCoat
# raw['ChestCoat'] = ["白毛" if x.find("白")!=-1 else x for x in raw['ChestCoat']] #496 objs
# raw['ChestCoat'] = ["黃毛" if x.find("黃")!=-1 else x for x in raw['ChestCoat']] #7 objs
# raw['ChestCoat'] = ["棕黑毛" if re.search('咖啡|黑|棕', x) else x for x in raw['ChestCoat']] #9 objs
# raw['ChestCoat'] = [x if (x == "無" or re.search('白毛|黃毛|棕黑毛', x)) else "其他" for x in raw['ChestCoat']]
# print("ChestCoat:")
# for i in set(raw['ChestCoat']):
#     print(i, sum([True if re.search(i, x) else False for x in raw['ChestCoat']]))
# print()

# #MouthCoat
# raw['MouthCoat'] = ["白" if x.find("白")!=-1 else x for x in raw['MouthCoat']] #117 objs
# raw['MouthCoat'] = ["黃" if x.find("黃")!=-1 else x for x in raw['MouthCoat']] #9 objs
# raw['MouthCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x) else x for x in raw['MouthCoat']] #242 objs
# raw['MouthCoat'] = [x if (x == "無" or re.search('白|黃|棕灰黑', x)) else "其他" for x in raw['MouthCoat']]
# print("MouthCoat:")
# for i in set(raw['MouthCoat']):
#     print(i, sum([True if re.search(i, x) else False for x in raw['MouthCoat']]))
# print()

# #HeadCoat
# raw['HeadCoat'] = ["虎斑" if re.search('虎斑|虎班', x) else x for x in raw['HeadCoat']] #23 objs
# raw['HeadCoat'] = ["黑白" if re.search('黑.*白', x) else x for x in raw['HeadCoat']] #4 objs
# raw['HeadCoat'] = ["棕灰黑" if (re.search('咖啡|黑|棕|灰', x) and x != "黑白") else x for x in raw['HeadCoat']] #41 objs
# raw['HeadCoat'] = ["受傷" if re.search('傷|浮腫|受傷|車禍|獨眼|白內障', x) else x for x in raw['HeadCoat']] #10 objs
# raw['HeadCoat'] = ["四眼" if re.search('四目|四眼', x) else x for x in raw['HeadCoat']] #66 objs
# raw['HeadCoat'] = ["白" if (x.find("白")!=-1 and x != "黑白") else x for x in raw['HeadCoat']] #18 objs
# raw['HeadCoat'] = ["黃" if x.find("黃")!=-1 else x for x in raw['HeadCoat']] #29 objs
# raw['HeadCoat'] = [x if (x == "無" or re.search('虎斑|黑白|棕灰黑|受傷|四眼|白|黃', x)) else "其他" for x in raw['HeadCoat']]
# print("HeadCoat:")
# for i in set(raw['HeadCoat']):
#     print(i, sum([True if re.search(i, x) else False for x in raw['HeadCoat']]))
# print()

# #FootCoat
# raw['FootCoat'] = ["受傷" if re.search('傷|截|不良於行|骨折|斷裂|缺|腫瘤|斷|脫臼|尺骨外露|跛行|三腳|後癱|癱瘓|畸形|短|無力', x) else x for x in raw['FootCoat']] #10 objs
# raw['FootCoat'] = ["虎斑" if re.search('虎斑|虎班|斑紋', x) else x for x in raw['FootCoat']] #23 objs
# raw['FootCoat'] = ["白" if x.find("白")!=-1 else x for x in raw['FootCoat']] #18 objs
# raw['FootCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x) else x for x in raw['FootCoat']] #23 objs
# raw['FootCoat'] = ["黃" if x.find("黃")!=-1 else x for x in raw['FootCoat']] #18 objs
# raw['FootCoat'] = [x if (x == "無" or re.search('虎斑|白|棕灰黑|受傷|黃', x)) else "其他" for x in raw['FootCoat']]
# print("FootCoat:")
# for i in set(raw['FootCoat']):
#     print(i, sum([True if re.search(i, x) else False for x in raw['FootCoat']]))
# print()

# #EarCoat
# raw['EarCoat'] = ["剪/缺/折/立/垂耳" if re.search('垂|剪|折|缺|立|摺|聳', x) else x for x in raw['EarCoat']] #10 objs
# raw['EarCoat'] = ["受傷" if re.search('左耳皺|皮膚炎|長瘤', x) else x for x in raw['EarCoat']] #23 objs
# raw['EarCoat'] = ["虎斑" if re.search('虎斑|虎班|斑紋', x) else x for x in raw['EarCoat']] #23 objs
# raw['EarCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x) else x for x in raw['EarCoat']] #23 objs
# raw['EarCoat'] = ["黃" if x.find("黃")!=-1 else x for x in raw['EarCoat']] #18 objs
# raw['EarCoat'] = [x if (x == "無" or re.search('虎斑|缺|棕灰黑|受傷|黃', x)) else "其他" for x in raw['EarCoat']]
# print("EarCoat:")
# for i in set(raw['EarCoat']):
#     print(i, sum([True if re.search(i, x) else False for x in raw['EarCoat']]))
# print()

# #BackCoat
# raw['BackCoat'] = ["受傷" if re.search('皮膚病|傷|突出物', x) else x for x in raw['BackCoat']]
# raw['BackCoat'] = ["斑點" if re.search('虎斑|虎班|斑紋|斑', x) else x for x in raw['BackCoat']]
# raw['BackCoat'] = ["白" if re.search('白', x) else x for x in raw['BackCoat']]
# raw['BackCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x)  else x for x in raw['BackCoat']] 
# raw['BackCoat'] = [x if (x == "無" or re.search('斑點|棕灰黑|受傷|白', x)) else "其他" for x in raw['BackCoat']]
# print("BackCoat:")
# for i in set(raw['BackCoat']):
#     print(i, sum([True if re.search(i, x) else False for x in raw['BackCoat']]))
# print()

# #TailCoat
# raw['TailCoat'] = ["有病" if re.search('皮膚病|肉瘤', x) else x for x in raw['TailCoat']]
# raw['TailCoat'] = ["截尾" if re.search('截|剪尾|半短|折尾|斷|短|半尾', x) else x for x in raw['TailCoat']]
# raw['TailCoat'] = ["無尾" if re.search('無尾', x) else x for x in raw['TailCoat']]
# raw['TailCoat'] = ["斑點" if re.search('虎斑|虎班|斑紋|斑', x) else x for x in raw['TailCoat']]
# raw['TailCoat'] = ["麒麟尾" if re.search('麒麟', x) else x for x in raw['TailCoat']]
# raw['TailCoat'] = ["鐮刀尾" if re.search('鐮刀', x) else x for x in raw['TailCoat']]
# raw['TailCoat'] = ["白" if re.search('白', x) else x for x in raw['TailCoat']]
# raw['TailCoat'] = ["棕灰黑" if re.search('咖啡|黑|棕|灰', x)  else x for x in raw['TailCoat']] 
# raw['TailCoat'] = [x if (x == "無" or re.search('斑點|棕灰黑|有病|白|截尾|無尾|麒麟尾|鐮刀尾', x)) else "其他" for x in raw['TailCoat']]
# print("TailCoat:")
# set(raw['TailCoat'])
# for i in set(raw['TailCoat']):
#     print(i, sum([True if re.search(i, x) else False for x in raw['TailCoat']]))
# print()

In [11]:
#有無項圈
raw['CollarName'] = ["無項圈" if x == "無" else "有項圈" for x in raw['CollarName']]
#有無衣服
raw['ThoracodorsalName'] = ["無衣服" if x == "無" else "有衣服" for x in raw['ThoracodorsalName']]

# NotAdoptionName有標注的欄位都刪掉？ //欄位留著

In [12]:
#每隻動物都被領養了嗎？？？因為有NotAdoptionName這個欄位
# print(set(raw['NotAdoptionName']))
# raw = raw[raw['NotAdoptionName']=='無']
# print("rm [NotAdoptionName], data shape", raw.shape)

# SituationRemark有標注的欄位都刪掉？ ////欄位刪掉

In [13]:
# print(set(raw['SituationRemark']))
# raw = raw[raw['SituationRemark']=='無']
# print("rm [SituationRemark], data shape", raw.shape)

# IsSterilizationName'填的有{'是', '未確定'}，填是的有14834筆，SterilizationDate 有填的9961筆


### 要保留IsSterilizationName這個欄位就好嗎？

In [14]:
# print(sum([True if x!='0001-01-01T00:00:00' else False for x in raw['SterilizationDate']]))
# print(sum([True if x=='是' else False for x in raw['IsSterilizationName']]))
# set(raw['SterilizationDate'])

#  備註欄位直接刪掉？內容大致為三類（病因、外觀、各種阿里阿咂），其中病因佔多數

In [15]:
# print(sum([True if x=='無' else False for x in raw['Note']]))
# set(raw['Note'])

In [16]:
# set(raw['HealthStatusName'])
# print(sum([True if x=='無' else False for x in raw['HealthStatusName']]))

#  健康狀態備註欄位直接刪掉？
- 絕育(佔多數)(紮)
- 病因or缺陷
- 疫苗（CCV\CPV\FIV\五合一）
- 驅蟲藥
- 幼犬奶貓

In [17]:
# set(raw['HealthStatusRemark'])
# print(sum([True if x=='無' else False for x in raw['HealthStatusRemark']]))

# [Behavior]可以有多種category，切成01表

In [18]:
tmp = []
for a in raw['Behavior']:
    tmp += [x for x in a.split(", ")]
health_types = set(tmp)
for i in health_types:
    health_types_col = []
    for a in raw['Behavior']:
        if i in a:
            health_types_col.append(1)
        else:
            health_types_col.append(0)
    raw['Behavior'+i] = health_types_col
raw.drop(columns=['Behavior','Behavior無'], inplace=True)

In [19]:
raw['SexName'].replace('未知', '無', inplace=True)
tmp = [True if x not in ["268", "269"] else False for x in raw['VarietyName'] ]
raw = raw[tmp]

In [20]:
file = open('clean1_description.txt', 'w')
file.write('# of columns:'+str(len(raw.columns)))
raw = raw.astype(str)
for col in raw.columns:
    if col in ['ＳInImage','ImageNameStr','FrontImageName']:
        continue
    print(col+' : ', set(raw[col]))
    file.write(col+' : '+', '.join(list(set(raw[col])))+'\n')
#     print(i, sum([True if re.search(i, x) else False for x in raw['TailCoat']]))
file.write("# of adopted data:{}".format(sum(raw['Adopted']=='1'))+'\n')
file.write("# of not adopted data:{}".format(sum(raw['Adopted']=='0'))+'\n')
file.close()
raw.to_csv('clean1.csv', index=False)

KeyNo :  {'76698', '56559', '44413', '53125', '59808', '101632', '51656', '49033', '47985', '141988', '37674', '97398', '107490', '153559', '97759', '121011', '157907', '28867', '139382', '58383', '79691', '123059', '84551', '90195', '71319', '36443', '153366', '92157', '116819', '37431', '121947', '92859', '132484', '47651', '50154', '45864', '120893', '103368', '126456', '143179', '65267', '159536', '151663', '151554', '157727', '77877', '54333', '101547', '62678', '75876', '91632', '83583', '140451', '75062', '132357', '30371', '59569', '99613', '142766', '143851', '157365', '88587', '152656', '137506', '111925', '98698', '135099', '146071', '58300', '78414', '40370', '135401', '54437', '44402', '142209', '149080', '142426', '101746', '96418', '98931', '134612', '155157', '113189', '104833', '100689', '139156', '49842', '67109', '81729', '95409', '118145', '46617', '129742', '35703', '62185', '29758', '82946', '67335', '41849', '34493', '53724', '90434', '62822', '108550', '138463',

HealthStatusName :  {'疫苗', '無', '正常', '異常', '寄生蟲', '其它'}
ShelterName :  {'臺南市動物之家善化站', '臺中市動物之家南屯園區', '嘉義市動物保護教育園區', '新北市八里區公立動物之家', '宜蘭縣流浪動物中途之家', '高雄市壽山動物保護教育園區', '臺北市動物之家', '新北市板橋區公立動物之家', '基隆市寵物銀行', '彰化縣流浪狗中途之家', '澎湖縣流浪動物收容中心', '屏東縣公立犬貓中途之家', '新北市中和區公立動物之家', '新北市瑞芳區公立動物之家', '新北市政府動物保護防疫處', '連江縣流浪犬收容中心', '新竹縣公立動物收容所', '南投縣公立動物收容所', '高雄市燕巢動物保護關愛園區', '新北市新店區公立動物之家', '雲林縣流浪動物收容所', '桃園市動物保護教育園區', '金門縣動物收容中心', '臺東縣動物收容中心', '無', '新北市三芝區公立動物之家', '花蓮縣流浪犬中途之家', '臺南市動物之家灣裡站', '苗栗縣生態保育教育中心(動物收容所)', '新竹市動物保護教育園區', '新北市淡水區公立動物之家', '新北市五股區公立動物之家', '嘉義縣流浪犬中途之家'}
Adopted :  {'1', '0'}
StayDays :  {'255', '23', '347', '781', '465', '228', '825', '724', '887', '917', '422', '489', '925', '858', '617', '1861', '200', '1080', '555', '1150', '893', '333', '683', '159', '157', '1076', '1065', '905', '892', '735', '1273', '603', '728', '155', '273', '337', '631', '977', '144', '216', '328', '78', '267', '615', '953', '210', '915', '256', '1215', '192', '277', '207', '238', '91', '285', '342', '477', '436'

In [21]:
print("# of adopted data:{}".format(sum(raw['Adopted']=='1')))
print("# of not adopted data:{}".format(sum(raw['Adopted']=='0')))

# of adopted data:62748
# of not adopted data:7997
